In [1]:
import numpy as np
import sys
import pickle
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from keras.utils import np_utils
from keras.layers import Dense
from keras.layers import LSTM
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint, Callback
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Activation

Using TensorFlow backend.


# Part 1: LSTM on txt data

# 1. concatenate all txt files into one corpus

In [2]:
path = 'Book Files/books/'
names = ['MLOE.txt','TPP.txt','OKEWFSMP.txt','TAM.txt','TAMatter.txt','THWP.txt']
output = open('corpus.txt', 'w')

for i in range(len(names)):
    file = path+names[i]
    with open(file, encoding='ascii', errors='ignore') as book:
        for line in book:
            output.write(line)

# 2. encode the characters in that file into range [0,1]

In [3]:
row_data = open('corpus.txt').read().lower()
chars = []
for n in row_data:
    chars.extend(list(set(set(n))))
chars = sorted(list(set(chars)))
chars_dict = dict((c, i) for i, c in enumerate(chars))
char_amount = len(row_data)
word_amount = len(chars)
int_dict = dict((i, c) for i, c in enumerate(chars))

# 4. set window size = 100 and make every first 99 in a window as input, output the 100 


In [4]:
# creat x as input and y as output
x=[]
y=[]
window_select = np.arange(0, char_amount-100, 1)
for i in window_select:
    input_range = row_data[i:i+100]
    output_word = row_data[i+100]
    y.append(chars_dict[output_word])
    temp=[]
    for k in input_range:
        temp.append(chars_dict[k]/float(word_amount))
    x.append(temp)
chunk_amout=len(x)

# 5. convert y from int type into catigorical data

In [5]:
y = np_utils.to_categorical(y)
x = np.reshape(x, (chunk_amout, 100, 1))

# 6. single layer LSTM

In [6]:
model = Sequential()
model.add(LSTM(256, input_shape=(x.shape[1], x.shape[2])))
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 256)               264192    
Total params: 264,192
Trainable params: 264,192
Non-trainable params: 0
_________________________________________________________________
None


# 7. softmax output layer LSTM

In [7]:
model = Sequential()
model.add(LSTM(256, input_shape=(x.shape[1], x.shape[2])))
model.add(Dense(y.shape[1], activation='softmax'))
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 256)               264192    
_________________________________________________________________
dense_1 (Dense)              (None, 67)                17219     
Total params: 281,411
Trainable params: 281,411
Non-trainable params: 0
_________________________________________________________________
None


# 10. select resonable epochs and test checkpoints

In [8]:
# using code from official documents: https://keras.io/zh/callbacks/#modelcheckpoint
model.compile(loss='categorical_crossentropy', optimizer='adam')
checkpointer = ModelCheckpoint(filepath='LSTMcheckpoints/weights.{epoch:02d}-{loss:.2f}.hdf5', monitor='loss', mode='min', verbose=0, save_best_only=True)
model.fit(x, y, batch_size=512, epochs=30, verbose=1, callbacks=[checkpointer])


Epoch 1/30
4346517/4346517 [==============================] - 894s 206us/step - loss: 2.6757
Epoch 2/30
4346517/4346517 [==============================] - 889s 204us/step - loss: 2.5553
Epoch 3/30
4346517/4346517 [==============================] - 886s 204us/step - loss: 2.3668
Epoch 4/30
4346517/4346517 [==============================] - 875s 201us/step - loss: 2.1658
Epoch 5/30
4346517/4346517 [==============================] - 879s 202us/step - loss: 2.0501
Epoch 6/30
4346517/4346517 [==============================] - 873s 201us/step - loss: 1.9735
Epoch 7/30
4346517/4346517 [==============================] - 874s 201us/step - loss: 1.9176
Epoch 8/30
4346517/4346517 [==============================] - 871s 200us/step - loss: 1.8748
Epoch 9/30
4346517/4346517 [==============================] - 873s 201us/step - loss: 1.8404
Epoch 10/30
4346517/4346517 [==============================] - 876s 202us/step - loss: 1.8118
Epoch 11/30
4346517/4346517 [==============================] - 878s 2

# 11. use the best loss weights, try to write like the author


In [10]:
model = Sequential()
model.add(LSTM(256,input_shape=(x.shape[1],x.shape[2])))
model.add(Dense(y.shape[1], activation='softmax'))
model.load_weights("LSTMcheckpoints/weights.21-1.68.hdf5")
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [24]:
initial = 'There are those who take mental phenomena naively, just as they would physical phenomena. This school of psychologists tends not to emphasize the object.'
initial = initial.lower()
chars = []
for n in initial:
    chars.append(chars_dict[n])
chars = chars[0:100]

In [25]:
for i in range(1000):
    x = np.reshape(chars, (1, len(chars), 1))
    x = x / float(word_amount)
    # get the predicted int value
    pred = model.predict(x, verbose=0)
    index = np.argmax(pred)
    # convert int to char
    result = int_dict[index]
    sys.stdout.write(result)

    # move window, drop the first and add the predict one
    chars.append(index)
    chars = chars[1:len(chars)]

a

the same tame the soarat of the point of view of the particular 
aetirativ that is the same tienry of shahet that the semsa oa 
she soace of the part of the part of the part of the poentsion 
of the point of view. and the soarates of the part of the 
semsa in which the semse of the part of the point of view of 
the part of the particular than the soace of the particular 
aetiraty of the particular enemgsh. the semoe of the semse of 
the eoem oo the part of the point of view that the semsa oa 
she soace of the part of the part of the part of the 
perception of the particular enemgsh. 

the semse of the part of the point of view of the semse of 
the part of the poentsion of the part of the point of view. 

the content of the part of the eoel oo the part of the point of view. the semoe of the 
consedot lenser of the part of the point of view of the particular enemgsh. the semoe of the 
consedot lenser the same aog to cemtarn the soace of the 
perception of the part of the poentsion of 